In [179]:
1+1

2

In [180]:
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt

 ### read and modify the data

In [197]:
dataset=pd.read_table("C:/Users/Gerald/Downloads/housing_prices.txt")

In [198]:
city=copy.deepcopy(dataset['City'])
city=city.drop_duplicates()
for i in city:
    dataset[str('city='+i+'?')]=(dataset['City']==i)*1
# Normalize each column
dataset['sqft_std']=(dataset['sqft']-dataset['sqft'].min()+0.0)/(dataset['sqft'].max()-dataset['sqft'].min())
dataset['price_std']=(dataset['price']-dataset['price'].min()+0.0)/(dataset['price'].max()-dataset['price'].min())
dataset['bedrooms_std']=(dataset['bedrooms']-dataset['bedrooms'].min()+0.0)/(dataset['bedrooms'].max()-dataset['bedrooms'].min())
dataset['bath_std']=(dataset['baths']-dataset['baths'].min()+0.0)/(dataset['baths'].max()-dataset['baths'].min())
dataset['bias']=1
dataset['rand']=np.random.uniform(0,1,len(dataset))

In [199]:
dataset=dataset.sort_index(by='rand')
dataset=dataset.drop(['rand'],axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


In [200]:
X=copy.deepcopy(dataset)
for i in ['sqft','price','City','bedrooms','baths','price_std']:
    X=X.drop([i],axis=1)

In [201]:
Y=dataset['price_std']

### Split the trainset and the testset

In [202]:
X_train=X[0:round(0.7*len(X))]
X_test=X[round(0.7*len(X)):len(X)]
Y_train=Y[0:round(0.7*len(X))]
Y_test=Y[round(0.7*len(X)):len(X)]

### Train the model

In [203]:
def cost_function(weight):
    A=Y_train-np.dot(weight, X_train.T )
    cost=1/(2*len(X_train))*sum([i*i for i in A])
    return cost

In [204]:
def gradient(weight):
    A=Y_train-np.dot(weight, X_train.T)
    grad=np.dot(A.T, X_train)/len(X_train)
    return -grad

In [205]:
r=0.01
theta=np.zeros(52)
grad=gradient(theta)
cost1=1
cost2=0
while cost1>cost2 and np.max(gradient(theta))>0.00001:
    cost1=cost_function(theta)
    theta=theta-r*gradient(theta)
    cost2=cost_function(theta)
theta=theta-r*gradient(theta)
print(theta)

[  0.00000000e+00   0.00000000e+00   5.33160043e-05   0.00000000e+00
   0.00000000e+00   3.12101513e-05   2.60233214e-05   5.33693588e-05
   5.20243499e-05   0.00000000e+00   1.72413793e-04   5.18831611e-05
   0.00000000e+00   8.93204807e-06   9.06729206e-05   0.00000000e+00
   8.16071147e-05   0.00000000e+00   8.17555859e-05   7.12037310e-05
   1.05386278e-04   3.56836063e-05   1.23384132e-04   6.09415360e-05
   8.29744166e-05   9.47005163e-05   1.92492324e-04   1.35110231e-04
   1.49972208e-04   6.68788957e-06   1.20099635e-04   5.35031166e-05
   3.99935796e-05   4.29659750e-05   1.00927685e-04   3.56836063e-05
   3.89532412e-05   1.78343722e-05   2.16002336e-04   1.38825725e-04
   1.84266220e-04   3.85073819e-05   4.59383704e-05   1.08522155e-04
   7.43098841e-06   3.77999518e-04   2.06730098e-05   9.05243008e-05
   1.58765102e-03   1.82991558e-03   1.20496316e-03   3.44039589e-03]


### test

In [206]:
residual=Y_train-np.dot(theta, X_train.T)
MSE=sum([i*i for i in residual])/len(residual)
print(MSE)

0.175762755871


## perception model

In [214]:
#set $500,000 as a threhold
c=(500000-dataset['price'].min())/(dataset['price'].max()-dataset['price'].min())
IsExp_test=(Y_test>c)*1
IsExp_train=(Y_train>c)*1

In [215]:
weight=np.zeros(52)

In [216]:
def predict(weight,Xi,y):
    M=np.dot(weight,Xi)
    if M>0:
        N=1
    else:
        N=0
    if N==y:
        return 1
    else:
        return 0

In [217]:
r=0.1
for loop in range(1000):
    wrong=0
    for i in X_train.index:
        if predict(weight,X_train.loc[i],IsExp_train[i])==1:
            pass
        else:
            weight=weight+r*(2*IsExp_train[i]-1)*X_train.loc[i]
            wrong=wrong+1
    if wrong==0:
        break
    else:
        pass
print(wrong)

0


In [218]:
IsExp_predict=copy.deepcopy(IsExp_test)
for i in IsExp_predict.index:
    IsExp_predict.loc[i]=(np.dot(theta,X_test.loc[i])>0)*1
correctRate=sum(IsExp_predict==IsExp_test)/len(IsExp_test)
print(correctRate)

0.4


In [219]:
loop

26

In [220]:
residual=Y_train-np.dot(weight, X_train.T)
MSE=sum([i*i for i in residual])/len(residual)
print(MSE)

0.185074974359
